In [4]:
#Spalte mit Übersetzer in .csv hinzufügen:
import pandas as pd

# Step 1: Read the original CSV
df = pd.read_csv('texte_mit_top3_sachgebieten.csv')

# Step 2: Read the Excel file and select the relevant column
# Assume the Excel file is 'values.xlsx', sheet is 'Sheet1', and column is 'NewValues'
excel_col = pd.read_excel('translators_only.xlsx', sheet_name='Sheet1')['Translator'] 

# Step 3: Assign the Excel column to the DataFrame as a new column
df['Translator'] = excel_col.values  # Make sure lengths match

# Step 4: Save the updated DataFrame to a new CSV
df.to_csv('texte_mit_top3_sachgebieten_translators.csv', index=False)




In [5]:
#Show the first 5 rows of new .csv
df = pd.read_csv('texte_mit_top3_sachgebieten_translators.csv')
print(df.head())


                             Dateiname  \
0       147053_IZA_fr-de.tmx_de-ch.txt   
1  150478_ZudokRng_de-fr.tmx_fr-ch.txt   
2       141610_IZA_fr-it.tmx_fr-ch.txt   
3  150183_ReqAvisC_fr-en.tmx_en-gb.txt   
4  146054_BegRng22_de-fr.tmx_fr-ch.txt   

                                                Text  \
0  Auswirkungen\r\nAuswirkungen auf den Bund\r\nF...   
1  Documentation complémentaire du DFAE\r\nCompte...   
2  Le DFAE et le DEFR considèrent que les critère...   
3  INTERNATIONAL COURT OF JUSTICE –\r\nOBLIGATION...   
4  Priorités stratégiques\r\nRelations avec les É...   

                 Top-Sachgebiet 1                Top-Sachgebiet 2  \
0  Entwicklung und Zusammenarbeit        Struktur und Institution   
1        Struktur und Institution  Entwicklung und Zusammenarbeit   
2  Entwicklung und Zusammenarbeit                         Politik   
3                           Recht  Entwicklung und Zusammenarbeit   
4        Struktur und Institution        Diplomatie und Protokoll

In [6]:
#Show the last 5 rows:
df = pd.read_csv('texte_mit_top3_sachgebieten_translators.csv')
print(df.iloc[:, -5:])

                                                  Text  \
0    Auswirkungen\r\nAuswirkungen auf den Bund\r\nF...   
1    Documentation complémentaire du DFAE\r\nCompte...   
2    Le DFAE et le DEFR considèrent que les critère...   
3    INTERNATIONAL COURT OF JUSTICE –\r\nOBLIGATION...   
4    Priorités stratégiques\r\nRelations avec les É...   
..                                                 ...   
116  22.xxx\r\nRapporto\r\ndel 18 maggio 2022\r\nOn...   
117  Entwurf vom 20. April 2023\r\nMaritime Strateg...   
118  Dieser Kredit dient der Finanzierung von klein...   
119  Conséquences\r\nConséquence pour la Confédérat...   
120  Excellency\r\nMadam, Sir\r\nSwitzerland and th...   

                   Top-Sachgebiet 1                Top-Sachgebiet 2  \
0    Entwicklung und Zusammenarbeit        Struktur und Institution   
1          Struktur und Institution  Entwicklung und Zusammenarbeit   
2    Entwicklung und Zusammenarbeit                         Politik   
3                  

In [3]:
#From BERTSimilar we need to use the bert-base-cased tokenizer to chunk the cells in column Text to no more than 512 tokens per row. We want to duplicate the values of the other columns for each chunk.
import pandas as pd
from transformers import AutoTokenizer

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Load your CSV
df = pd.read_csv("texte_mit_top3_sachgebieten_translators.csv") 




In [14]:
# Silence the warning from the transformers library I got in the beginning:
from transformers import logging
logging.set_verbosity_error()

# Columns to keep for each chunk
metadata_cols = ["Dateiname", "Top-Sachgebiet 1", "Top-Sachgebiet 2", "Top-Sachgebiet 3", "Translator"]

# Store all chunked rows
expanded_rows = []

# Iterate through each row of the DataFrame
for idx, row in df.iterrows():
    text = row["Text"]
    tokens = tokenizer.tokenize(text)

    # Chunk into segments of 512 tokens
    for chunk_id, i in enumerate(range(0, len(tokens), 512), start=1):
        chunk_tokens = tokens[i:i + 512]
        chunk_text = tokenizer.convert_tokens_to_string(chunk_tokens)
        #chunk_text = "".join([
    #token if not token.startswith("##") else token[2:]
    #if token != "##" else ""
    #for token in chunk_tokens
#])

        new_row = {col: row[col] for col in metadata_cols}
        new_row["Text"] = chunk_text
        new_row["original_row_id"] = idx
        new_row["chunk_id"] = chunk_id
        expanded_rows.append(new_row)

# Convert to new DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# Optional: Save to file
expanded_df.to_csv("texte_mit_top3_sachg_tr_chunked_with_ids.csv", index=False)

In [16]:
# Read and preview the output
df_chunked = pd.read_csv("texte_mit_top3_sachg_tr_chunked_with_ids.csv")
print(df_chunked.head())           # See the first 5 rows
print(df_chunked.shape)           # Rows and columns
print(df_chunked.columns.tolist()) # Column names

                             Dateiname                Top-Sachgebiet 1  \
0       147053_IZA_fr-de.tmx_de-ch.txt  Entwicklung und Zusammenarbeit   
1       147053_IZA_fr-de.tmx_de-ch.txt  Entwicklung und Zusammenarbeit   
2       147053_IZA_fr-de.tmx_de-ch.txt  Entwicklung und Zusammenarbeit   
3       147053_IZA_fr-de.tmx_de-ch.txt  Entwicklung und Zusammenarbeit   
4  150478_ZudokRng_de-fr.tmx_fr-ch.txt        Struktur und Institution   

                 Top-Sachgebiet 2  Top-Sachgebiet 3 Translator  \
0        Struktur und Institution             Recht        kas   
1        Struktur und Institution             Recht        kas   
2        Struktur und Institution             Recht        kas   
3        Struktur und Institution             Recht        kas   
4  Entwicklung und Zusammenarbeit  Humanitäre Hilfe        bec   

                                                Text  original_row_id  \
0  Auswirkungen Auswirkungen auf den Bund Finanzi...                0   
1  ##e Zusam

In [17]:
#Check how a long document got split:
df_chunked[df_chunked["original_row_id"] == 2]


,Dateiname,Top-Sachgebiet 1,Top-Sachgebiet 2,Top-Sachgebiet 3,Translator,Text,original_row_id,chunk_id
36,141610_IZA_fr-it.tmx_fr-ch.txt,Entwicklung und Zusammenarbeit,Politik,Struktur und Institution,chm,Le DFAE et le DEFR considèrent que les critère...,2,1
37,141610_IZA_fr-it.tmx_fr-ch.txt,Entwicklung und Zusammenarbeit,Politik,Struktur und Institution,chm,"de l ’ engagement en Afrique subsaharienne, 1 ...",2,2
38,141610_IZA_fr-it.tmx_fr-ch.txt,Entwicklung und Zusammenarbeit,Politik,Struktur und Institution,chm,##me pertinente par 6 associations et 3 autres...,2,3
39,141610_IZA_fr-it.tmx_fr-ch.txt,Entwicklung und Zusammenarbeit,Politik,Struktur und Institution,chm,##ique proposée pertinente. 10 estiment qu ’ u...,2,4
40,141610_IZA_fr-it.tmx_fr-ch.txt,Entwicklung und Zusammenarbeit,Politik,Struktur und Institution,chm,Réponses Question 3 : Vue d ' ensemble des rép...,2,5
41,141610_IZA_fr-it.tmx_fr-ch.txt,Entwicklung und Zusammenarbeit,Politik,Struktur und Institution,chm,Soutenez - vous l ’ allocation des ressources ...,2,6
42,141610_IZA_fr-it.tmx_fr-ch.txt,Entwicklung und Zusammenarbeit,Politik,Struktur und Institution,chm,", ZH 9 ( 39 % ) AG, TI 2 ( 9 % ) 12 ( 52 % ) 7...",2,7
43,141610_IZA_fr-it.tmx_fr-ch.txt,Entwicklung und Zusammenarbeit,Politik,Struktur und Institution,chm,##ne Mittelzuweisung für die Ukraine? Ja Nein ...,2,8
44,141610_IZA_fr-it.tmx_fr-ch.txt,Entwicklung und Zusammenarbeit,Politik,Struktur und Institution,chm,que ces fonds soient comptabilisés comme des b...,2,9
45,141610_IZA_fr-it.tmx_fr-ch.txt,Entwicklung und Zusammenarbeit,Politik,Struktur und Institution,chm,##les estimant que ce soutien nécessite plus d...,2,10


In [18]:
#Add "Top-3 alle" column between "Top-Sachgebiet 3" and "Translator", with the values from the three previous columns:
import pandas as pd

# Load your CSV (replace with your actual file name)
df = pd.read_csv("texte_mit_top3_sachg_tr_chunked_with_ids.csv")

# Create the new column content
df["Top-3 alle"] = (
    "Die Top 3 Sachgebiete in diesem Text sind: " +
    df["Top-Sachgebiet 1"].astype(str) + ", " +
    df["Top-Sachgebiet 2"].astype(str) + ", " +
    df["Top-Sachgebiet 3"].astype(str)
)

# Reorder columns to insert "Top-3 alle" between "Top-Sachgebiet 3" and "Translator"
cols = df.columns.tolist()
insert_at = cols.index("Translator")
cols = cols[:insert_at] + ["Top-3 alle"] + cols[insert_at:]
df = df[cols]

# Save to new CSV
df.to_csv("texte_mit_top3_sachg_tr_chunked_with_ids_top3.csv", index=False)


In [22]:
# Anzeigen, wie das neue .csv aussieht:
#Load the new file
df_new = pd.read_csv("texte_mit_top3_sachg_tr_chunked_with_ids_top3.csv")

# Get the first 3 unique original_row_ids
first_three_ids = df_new["original_row_id"].unique()[:3]

# Filter rows where original_row_id is one of those 3
filtered_df = df_new[df_new["original_row_id"].isin(first_three_ids)]

# Display it
print(filtered_df)


                              Dateiname                Top-Sachgebiet 1  \
0        147053_IZA_fr-de.tmx_de-ch.txt  Entwicklung und Zusammenarbeit   
1        147053_IZA_fr-de.tmx_de-ch.txt  Entwicklung und Zusammenarbeit   
2        147053_IZA_fr-de.tmx_de-ch.txt  Entwicklung und Zusammenarbeit   
3        147053_IZA_fr-de.tmx_de-ch.txt  Entwicklung und Zusammenarbeit   
4   150478_ZudokRng_de-fr.tmx_fr-ch.txt        Struktur und Institution   
..                                  ...                             ...   
66       141610_IZA_fr-it.tmx_fr-ch.txt  Entwicklung und Zusammenarbeit   
67       141610_IZA_fr-it.tmx_fr-ch.txt  Entwicklung und Zusammenarbeit   
68       141610_IZA_fr-it.tmx_fr-ch.txt  Entwicklung und Zusammenarbeit   
69       141610_IZA_fr-it.tmx_fr-ch.txt  Entwicklung und Zusammenarbeit   
70       141610_IZA_fr-it.tmx_fr-ch.txt  Entwicklung und Zusammenarbeit   

                  Top-Sachgebiet 2          Top-Sachgebiet 3  \
0         Struktur und Institution 